In [2]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [3]:
df = pd.read_csv('Online Sales Data.csv')

(0) Some minor data preprocessing

In [4]:
df['Date'] = pd.to_datetime(df['Date'])  # Ensure the Date column is in datetime format
df['Month'] = df['Date'].dt.month  # Extract the month
df['Day Name'] = df['Date'].dt.strftime('%A')
df

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,Region,Payment Method,Month,Day Name
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card,1,Monday
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal,1,Tuesday
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card,1,Wednesday
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card,1,Thursday
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal,1,Friday
...,...,...,...,...,...,...,...,...,...,...,...
235,10236,2024-08-23,Home Appliances,Nespresso Vertuo Next Coffee and Espresso Maker,1,159.99,159.99,Europe,PayPal,8,Friday
236,10237,2024-08-24,Clothing,Nike Air Force 1 Sneakers,3,90.00,270.00,Asia,Debit Card,8,Saturday
237,10238,2024-08-25,Books,The Handmaid's Tale by Margaret Atwood,3,10.99,32.97,North America,Credit Card,8,Sunday
238,10239,2024-08-26,Beauty Products,Sunday Riley Luna Sleeping Night Oil,1,55.00,55.00,Europe,PayPal,8,Monday


(1) Total Sales over time by Regions

In [5]:
#Define Panel Widget
month_slider = pn.widgets.IntSlider(name='Month Slider', start= 1, end=8, step=1, value=8)
# Make DataFrame Pipeline Interactive
idf = df.interactive()

In [6]:
regions = df['Region'].unique().tolist()
regions

['North America', 'Europe', 'Asia']

In [7]:
revenue_pipeline = (
    idf[
        (idf.Month <= month_slider) &
        (idf['Region'].isin(regions))
    ]
    .groupby(['Region', 'Month'])['Total Revenue'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by = 'Month')
    .reset_index(drop = True)
)
revenue_plot = revenue_pipeline.hvplot(x = 'Month', by = 'Region', y = 'Total Revenue', line_width = 2, title = 'Total Revenue over months by Regions')
revenue_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'b92560da-0eee-417c-ac91-67315821ca80': {'version…

(2) Table - Total Sales Overtime by Regions

In [8]:
revenue_table = revenue_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
revenue_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'ae3ef4e1-7bad-4cc8-8ec1-6c853d9cff61': {'version…

(3) Total Revenue by Date

In [9]:
revenue_date = df.groupby('Date')['Total Revenue'].sum().hvplot(grid=True, figsize=(8, 6), title='Total Revenue Over Time', xlabel='Date', ylabel='Total Revenue')
revenue_date

:Curve   [Date]   (Total Revenue)

(4) Barchart : Unit sales by Product Category

In [10]:
day_name = df['Day Name'].unique().tolist()
products = df['Product Category'].unique().tolist()

day_select = pn.widgets.Select(name='Day Select', options= day_name)
day_select

BokehModel(combine_events=True, render_bundle={'docs_json': {'efc91228-2d1e-4a93-b79b-2285a7f1579a': {'version…

In [11]:
unit_sale_pipeline = (
    idf[
        (idf['Day Name'] == day_select) &
        idf['Product Category'].isin(products)
    ]
    .groupby(['Day Name', 'Product Category'])['Units Sold'].sum()  # Corrected line
    .to_frame()
    .reset_index()
    .reset_index(drop = True)
)
unit_sale_pipeline

BokehModel(combine_events=True, render_bundle={'docs_json': {'1ab9541b-3ece-419c-843f-e06d79338a83': {'version…

In [12]:
unit_plot = unit_sale_pipeline.hvplot(x = 'Product Category', y = 'Units Sold', kind = 'bar', title = 'Unit Sold by Product Categories Over Day', rot = 45, figsize=(8, 6))
unit_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'79ffd45c-99b5-4147-8e42-aa4136ff2837': {'version…

Creating Dashboard

In [16]:
template = pn.template.FastListTemplate(
    title='Online Sales Data dashboard', 
    sidebar=[pn.pane.Markdown("# Online Sales Data"), 
             pn.pane.Markdown("This dataset provides a comprehensive overview of online sales transactions across different product categories. Each row represents a single transaction with detailed information such as the order ID, date, category, product name, quantity sold, unit price, total price, region, and payment method."),
             month_slider
             ],
    main=[pn.Row(pn.Column(
                           revenue_plot, margin=(0,25)), 
                 revenue_table.panel(width=500)), 
          pn.Row(pn.Column(revenue_date, margin=(0,25)), 
                 pn.Column(day_select, unit_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();